In [1]:
from osgeo import gdal
import numpy as np

In [39]:
count = 0

In [40]:
def crop_by_slide_window(image_path, output_folder, window_size, step_size, image_class, save_as_array=False):
    global count

    ds = gdal.Open(image_path)
    if ds is None:
        return False

    # 获取图像的宽度和高度
    width = ds.RasterXSize
    height = ds.RasterYSize

    # 计算图像平均值、标准差
    whole_image = ds.ReadAsArray()
    mean = np.mean(whole_image.reshape(whole_image.shape[0], -1), axis=1)
    std = np.std(whole_image.reshape(whole_image.shape[0], -1), axis=1)

    # 计算滑动窗口的数量
    num_windows_width = (width - window_size) // step_size + 1
    num_windows_height = (height - window_size) // step_size + 1

    # 对每个滑动窗口进行裁剪
    for i in range(num_windows_width):
        for j in range(num_windows_height):
            # 计算滑动窗口的起始坐标
            start_x = i * step_size
            start_y = j * step_size

            # 读取滑动窗口中的数据
            image = ds.ReadAsArray(start_x, start_y, window_size, window_size)

            # MODIS产品无需去异常值
            if image_class != "MODIS":
                # 3σ法则去除异常值
                for band in range(image.shape[0]):
                    image[band, :, :] = np.clip(image[band, :, :], mean[band]-3*std[band], mean[band]+3*std[band])
            
            if save_as_array:
                if image_class == "S2":
                    selected_bands = [1, 2, 3, 4, 7, 8, 10, 11]
                elif image_class == "S1":
                    selected_bands = [0, 1]
                elif image_class == "MODIS":
                    selected_bands = [0, 1, 2, 3, 5, 6]

                image = image[selected_bands, :, :]
                # S2: (8, 250, 250), S1: (2, 250, 250), MODIS: (6, 5, 5)
                np.save(output_folder + image_class + f"_{count}.npy", image)

            else:
                # 创建新的tif文件来保存裁剪后的图像
                driver = gdal.GetDriverByName('GTiff')
                out_ds = driver.Create(output_folder + image_class + f"_{count}.tif", window_size, window_size, ds.RasterCount, gdal.GDT_Int16)

                # 将裁剪后的图像数据写入新的tif文件
                for k in range(ds.RasterCount):
                    out_band = out_ds.GetRasterBand(k + 1)
                    out_band.WriteArray(image[k])

                # 设置新tif文件的地理变换参数和投影信息
                geo_transform = list(ds.GetGeoTransform())
                geo_transform[0] += start_x * geo_transform[1]
                geo_transform[3] += start_y * geo_transform[5]
                out_ds.SetGeoTransform(geo_transform)
                out_ds.SetProjection(ds.GetProjection())

                # 关闭新tif文件
                out_ds = None

            count += 1

    # 关闭原始tif文件
    ds = None

    return True

In [41]:
import pandas as pd

xlsx_path = r"D:\Code\MODIS_S1_S2\dataset\数据时间匹配.xlsx"

# 读取xlsx文件
df = pd.read_excel(xlsx_path)

S2_date = [str(date) for date in df['S2'].tolist()]
MODIS_date = [str(date) for date in df['MODIS'].tolist()]
S1_date = [str(date) for date in df['S1'].tolist()]

S2_filename = [r"D:\ENVI\data\NingBo\NB_roi_S2\well_image\image\NB_S2_" + date + ".tif" for date in S2_date]
MODIS_filename = [r"D:\ENVI\data\NingBo\NB_roi_MODIS\MODIS_images\NB_MODIS_" + date + ".tif" for date in MODIS_date]
S1_filename = [r"D:\ENVI\data\NingBo\NB_roi_S1\S1_images\NB_S1_" + date + ".tif" for date in S1_date]


In [42]:
def batch_crop_image(image_filename_list, output_folder, window_size, step_size, image_class):
    global count
    count = 0
    for i, image_path in enumerate(image_filename_list):
        if not crop_by_slide_window(image_path, output_folder, window_size, step_size, image_class, save_as_array=True):
            print("fail to open " + image_path)
            count = 0
            return
    count = 0
    print("accomplish!")

In [43]:
# Sentinel-2
batch_crop_image(S2_filename, output_folder=r"D:\Code\MODIS_S1_S2\dataset\Sentinel-2\\", window_size=250, step_size=150, image_class="S2")

accomplish!


In [44]:
# Sentinel-1
batch_crop_image(S1_filename, output_folder=r"D:\Code\MODIS_S1_S2\dataset\Sentinel-1\\", window_size=250, step_size=150, image_class="S1")

accomplish!


In [31]:
# MODIS
batch_crop_image(MODIS_filename, output_folder=r"D:\Code\MODIS_S1_S2\dataset\MODIS\\", window_size=5, step_size=3, image_class="MODIS")

accomplish!


In [45]:
# crop_by_slide_window(S2_filename[0], r"D:\Code\MODIS_S1_S2\dataset\cropped_image\\", 250, 150, "S2")

True

In [16]:
image = np.load("../dataset/Sentinel-2/S2_100.npy")

min_val = np.min(image)
max_val = np.max(image)
mean = np.mean(image.reshape(-1))
std = np.std(image.reshape(-1))

print(min_val, max_val, mean, std)

0.014568317742677504 0.7682870725348873 0.1946172994939426 0.08272557610078235


In [46]:
for i in range(1512):
    image = np.load("../dataset/Sentinel-2/S2_" + str(i) + ".npy")
    min_val = np.min(image)
    max_val = np.max(image)
    if  min_val  == 1:
        print(i)

106
107
109
110
111
112
113
114
115
116
117
118
119
123
124
143
146
228
229
267
268
270
271
282
300
301
304
311
312
313
314
477
479
480
482
533
536
546
548
549
551
560
563
566
567
568
569
572
578
581
584
585
586
638
641
711
775
776
882
883
884
886
887
888
889
890
891
894
895
896
897
898
899
900
901
902
955
984
985
987
988
990
991
1142
1145
